In [ ]:
import numpy as np
import cvxpy as cp
import networkx as nx

from numpy.random import default_rng
from opt_utils import decompose_psd, hyperplane_rounding, complex_hyperplane_rounding, fixed_point_iteration, normalize_rows, load_graph

# Max-cut

In [ ]:
# graph_file = "torusg3-8.dat"
graph_file = "toruspm3-8-50.dat"

n = 100
G = load_graph(graph_file, n)

In [ ]:
L = nx.laplacian_matrix(G).toarray() * 1.0

In [ ]:
X = cp.Variable((n,n), PSD=True)
constraints = [ X[i][i] == 1 for i in range(n) ]
prob = cp.Problem(cp.Maximize(1/4 * (cp.trace(L @ X))), constraints)
prob.solve()

In [ ]:
_, x_0 = hyperplane_rounding(decompose_psd(X.value), lambda Y : -1/4 * np.trace(Y @ Y.T @ L), 0, n, 1)

In [ ]:
# x = cp.Variable(n)
# constraints = [ cp.square(x[i]) == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.norm(x - x_0.T[0])), constraints)
# prob.solve()

In [ ]:
X_close = cp.Variable((n,n), PSD=True)
constraints = [ X_close[i][i] == 1 for i in range(n) ]
prob = cp.Problem(cp.Maximize(cp.sum([x_0[i] * X_close[i][0] for i in range(n)])), constraints)
prob.solve(solver=cp.MOSEK)

In [ ]:
a = np.sign(x_0)
aa = a @ a.T
np.sum([x_0[i] * aa[i][0] for i in range(n)])

In [ ]:
x_0

In [ ]:
X_close.value

In [ ]:
1/4 * np.trace(X_close.value @ L)

In [ ]:
fixed_point_iteration(prob, X, np.zeros((n,n)), False)

# PhaseCut (maximize)

In [ ]:
n = 50  # number of observations
p = 20  # dimension of x
max_val = 10

In [ ]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-9) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

In [ ]:
U = cp.Variable((n,n), hermitian=True)

### Unit modulus

In [ ]:
constraints = [U >> 0]
constraints += [U[i][i] == 1 for i in range(n)]

prob = cp.Problem(cp.Maximize(cp.real(cp.trace(M @ U))), constraints)
prob.solve()

In [ ]:
complex_hyperplane_rounding(decompose_psd(U.value), lambda u : -np.real(u.conj().T @ M @ u))

In [ ]:
fixed_point_iteration(prob, U, -np.zeros((n,n)), True)

### Relaxed bounds

In [ ]:
delta = 0.05

In [ ]:
constraints = [U >> 0]
constraints += [cp.real(U[i][i]) >= 1 - delta for i in range(n)]
constraints += [cp.real(U[i][i]) <= 1 + delta  for i in range(n)]

prob = cp.Problem(cp.Maximize(cp.real(cp.trace(M @ U))), constraints)
prob.solve()

In [ ]:
complex_hyperplane_rounding(decompose_psd(U.value), lambda u : -np.real(u.conj().T @ M @ u), 1 - delta, 1 + delta)

In [ ]:
# fixed_point_iteration(prob, U, np.zeros((n,n)), True)
fixed_point_iteration(prob, U, -np.eye(n), True)